In [28]:
from peft import LoraConfig, TaskType
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [43]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [44]:
tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [4]:
mdl = AutoModelForSeq2SeqLM.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
from peft import get_peft_model

In [7]:
mdl = get_peft_model(mdl, peft_config)
mdl.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'google-t5/t5-small' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [16]:
from transformers import TrainingArguments, Trainer

In [17]:
MDL_PATH = Path.cwd()/'models' ; MDL_PATH
bs = 4

Loading our synthetic dataset

In [19]:
DS_PATH = Path.cwd()/'data'; DS_PATH

PosixPath('/teamspace/studios/this_studio/data')

In [22]:

# Load the data
df = pd.read_csv(DS_PATH/'synth-itr1-n166.csv')

# Display class distribution
print(df['label'].value_counts())

label
Negative      122
Positive       34
Not Stated     10
Name: count, dtype: int64


In [23]:
# Encode labels
label_encoding = {'Positive': 0, 'Negative': 1, 'Not Stated': 2}

In [24]:
# Balance the dataset
def balance_dataset(df):
    min_class_count = df['label'].value_counts().min()
    balanced_df = df.groupby('label').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)
    return balanced_df

balanced_df = balance_dataset(df)
print(balanced_df['label'].value_counts())

label
Negative      10
Not Stated    10
Positive      10
Name: count, dtype: int64


In [25]:
balanced_df.head()

,Unnamed: 0,report_text,label
0,35,BILATERAL SCREENING MAMMOGRAM\nHistory: Screen...,Negative
1,84,BILATERAL DIAGNOSTIC MAMMOGRAM. History: Patie...,Negative
2,87,BILATERAL DIAGNOSTIC MAMMOGRAM. History: Scree...,Negative
3,85,BILATERAL DIAGNOSTIC MAMMOGRAM - FOR SCREENING...,Negative
4,52,BILATERAL SCREENING MAMMOGRAPHY\nHistory: Scre...,Negative


In [26]:
balanced_df['label_encoded'] = balanced_df['label'].map(label_encoding)

In [29]:

# Split the data
train_df, temp_df = train_test_split(balanced_df, test_size=0.3, stratify=balanced_df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

In [33]:
from datasets import Dataset

In [52]:
def tok_func(x): return tokenizer(x['report_text'], truncation=True, padding='max_length', max_length=512)

In [53]:
train_dataset = Dataset.from_pandas(train_df).map(tok_func, batched=True)
val_dataset = Dataset.from_pandas(val_df).map(tok_func, batched=True)
test_df = Dataset.from_pandas(test_df).map(tok_func, batched=True)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'columns'

In [ ]:
train_dataset[0]['input']

"BREAST MAGNETIC RESONANCE IMAGING\n\nClinical History: Patient with history of right breast cancer, status post lumpectomy and radiation therapy 2 years ago, now presents with new microcalcifications on screening mammogram in the right breast.\n\nComparison: Prior mammograms.\n\nFindings:\nRight Breast: There is a small cluster of punctate enhancing foci in the right breast at the 3 o'clock position, corresponding to the microcalcifications seen on mammography. These foci demonstrate rapid uptake and washout kinetics, suggestive of suspicious enhancement. No other suspicious masses or areas of enhancement are seen in either breast.\n\nImpression:\nThe enhancing foci in the right breast corresponding to the new microcalcifications are suspicious for recurrent malignancy. Correlation with mammographic findings and biopsy are recommended.\n\nBI-RADS Category: 4 (Suspicious for Malignancy)"

In [ ]:
train_ds = 

In [18]:
training_args = TrainingArguments(
    output_dir=MDL_PATH,
    learning_rate=1e-3,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=mdl,
    args=training_args,
    train_dataset=
)